In [1]:
from __future__ import print_function

import numpy as  np
import tensorflow  as tf
from tensorflow.contrib import rnn


/home/dath/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
params = {
    'learning_rate': 0.001,
    'training_steps': 20000,
    'batch_size': 128,
    'display_step': 100,
    
    'num_input': 28,
    'timesteps': 28,
    'num_hidden': 128,
    'num_classes': 10
}

In [3]:
type(tf.estimator.ModeKeys.EVAL)

str

In [4]:
def estimator_spec(logits, labels, mode, params):

    
    
    loss, train_op, predictions = None, None, None
    
    if mode == tf.estimator.ModeKeys.EVAL:
        onehot_labels = tf.one_hot(labels, params['num_classes'], 1, 0)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=onehot_labels))
        metrics =  {'accuracy': accuracy}
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=params['learning_rate'])
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {'prediction': tf.argmax(logits, 1),'prob': testimator_specf.nn.softmax(logits)}
        eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=tf.argmax(logits, 1))}
        print(eval_metric_ops)
    
        return tf.estimator.EstimatorSpec(mode=mode,loss= loss, train_op= train_op, eval_metric_ops=eval_metric_ops)
#     return tf.estimator.EstimatorSpec(
#         mode=mode,
#         predictions=predictions,
#         loss=loss,
#         train_op=train_op,
#         eval_metric_ops=eval_metric_ops)

In [5]:
def model_fn(features, labels, mode, params):
    '''
    To classify images using a recurrent neural network, we consider every image
    row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
    handle 28 sequences of 28 steps for every sample. RNN
    '''
    x = tf.reshape(features['x'], [-1, params['timesteps'], params['num_input']])
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, params['timesteps'], 1)
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(params['num_hidden'], forget_bias=1.0)
    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    # outputs, states = tf.nn.dynamic_rnn(lstm_cell, x, dtype=tf.float32)
    # Linear activation, using rnn inner loop last output
    logits = tf.layers.dense(states[-1], units=params['num_classes'])
    return estimator_spec(logits=logits, labels=labels, mode=mode, params=params)

In [6]:
def main(args=None):
    run_config = tf.estimator.RunConfig()
    # run_config = run_config.replace(**{'save_checkpoints_steps': 100, 'keep_check point_max': 20})
    classifier = tf.estimator.Estimator(model_fn=model_fn,
                                        model_dir='./checkpoint',
                                        params=params,
                                        config=run_config)
    # make data
    mnist = tf.contrib.learn.datasets.DATASETS['mnist']('/tmp/mnist')
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': mnist.train.images},
        y=mnist.train.labels.astype(np.int32),
        batch_size=params['batch_size'],
        num_epochs=None,
        shuffle=True)
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': mnist.train.images},
        y=mnist.train.labels.astype(np.int32),
        num_epochs=1,
        shuffle=False)
    # classifier.train(input_fn=train_input_fn, steps=params['training_steps'], hooks=[])
    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=params['training_steps'])
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)


    tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)
    score = classifier.evaluate(eval_input_fn, steps=1)
    print(score)

In [7]:
if __name__ == '__main__':
    main(None)

INFO:tensorflow:Using config: {'_model_dir': './checkpoint', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0f1c1266d8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz
INFO:tensorf

AttributeError: 'NoneType' object has no attribute 'dtype'

In [ ]:
""" Recurrent Neural Network.
A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Links:
    [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf)
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Step 1, Minibatch Loss= 2.9108, Training Accuracy= 0.117
Step 200, Minibatch Loss= 2.1062, Training Accuracy= 0.203
Step 400, Minibatch Loss= 1.8828, Training Accuracy= 0.391
Step 600, Minibatch Loss= 1.8135, Training Accuracy= 0.383
Step 800, Minibatch Loss= 1.6374, Training Accuracy= 0.484
Step 1000, Minibatch Loss= 1.6117, Training Accuracy= 0.477
Step 1200, Minibatch Loss= 1.5341, Training Accuracy= 0.547
Step 1400, Minibatch Loss= 1.4050, Training Accuracy= 0.555
Step 1600, Minibatch Loss= 1.3843, Training Accuracy= 0.508
Step 1800, Minibatch

ModuleNotFoundError: No module named 'iris_data'